In [1]:
import glob
import numpy as np
import pandas as pd
import scipy.signal as sig

In [2]:
files = map(np.load, glob.glob("../export/data/gaze_data_*.npz"))

In [3]:
sessions_to_skip = {
    30   : [800, 1178, 1712, 2628, 3646],
    53   : [426, 814, 3028, 3538],
    69   : [1726, 2866, 3134, 3670],
    90   : [664, 932, 1714, 3012],
    111  : [32, 1210, 2354, 3158]
}

def resample(data, old_fps, new_fps):
    new_size = new_fps * data.size / old_fps
    return sig.resample(data, int(new_size))

def export_gaze_data(mediafile):

    valid_files = [ 
        f
        for f in files 
        if f["session_info"].tolist().get("mediaFile") == "{}.avi".format(mediafile) 
    ]

    sinfo = []
    coordinates_data = []
    fixations_data = []
    coordinates_data_resampled = []
    fixations_data_resampled = []

    for f in valid_files:
            sid = f["session_info"].tolist().get("sessionId")
            if sid not in sessions_to_skip[mediafile]:
                sinfo.append(f["session_info"].tolist())
                
                coordinates = f["coordinates"]
                coordinates_resampled = resample(coordinates, 60, 24)
                coordinates_data.append(coordinates)
                coordinates_data_resampled.append(coordinates_resampled)
                
                fixations = f["fixations"]
                fixations_resampled = resample(fixations, 60, 24)
                fixations_data.append(fixations)
                fixations_data_resampled.append(fixations_resampled)

    coordinates_data = np.array(zip(*coordinates_data))
    coordinates_data_resampled = np.array(zip(*coordinates_data_resampled))
    fixations_data = np.array(zip(*fixations_data))
    fixations_data_resampled = np.array(zip(*fixations_data_resampled))
    
    np.save("../export/sinfo_without_bad_session_{}.npy".format(mediafile), sinfo)
    np.save("../export/coordinates_data_without_bad_session_{}.npy".format(mediafile), coordinates_data)
    np.save("../export/coordinates_data_resampled_without_bad_session_{}.npy".format(mediafile), coordinates_data_resampled)
    np.save("../export/fixations_data_without_bad_session_{}.npy".format(mediafile), fixations_data)
    np.save("../export/fixations_data_resampled_without_bad_session_{}.npy".format(mediafile), fixations_data_resampled)
    
    del coordinates_data
    del coordinates_data_resampled
    del fixations_data
    del fixations_data_resampled

In [4]:
for mediafile in [30, 53, 69, 90, 111]:
    export_gaze_data(mediafile=mediafile)